<a href="https://colab.research.google.com/github/merajsiddieque/ML-Lab/blob/main/ML_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [25]:
df = pd.read_csv("/content/Housing.csv")
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [26]:
df.isnull().sum()

,0
price,0
area,0
bedrooms,0
bathrooms,0
stories,0
mainroad,0
guestroom,0
basement,0
hotwaterheating,0
airconditioning,0


In [21]:
df['furnishingstatus'].unique()


array(['furnished', 'semi-furnished', 'unfurnished'], dtype=object)

In [29]:
min_val = df['price'].min()
max_val = df['price'].max()

df['price_normalized'] = (df['price'] - min_val) / (max_val - min_val)

df['price_normalized']


,price_normalized
0,1.000000
1,0.909091
2,0.909091
3,0.906061
4,0.836364
...,...
540,0.006061
541,0.001485
542,0.000000
543,0.000000
